# PDF 문제를 JSON으로 변환

정보처리기사 PDF 파일을 읽어서 구조화된 JSON 파일로 변환합니다.

## 1. 라이브러리 import

In [ ]:
import pdfplumber
import json
import re
import os

## 2. PDF 파일 읽기

In [ ]:
def read_pdf_text(file_path):
    """PDF 파일의 모든 텍스트를 읽어옵니다."""
    all_text = []
    
    with pdfplumber.open(file_path) as pdf:
        print(f"총 페이지 수: {len(pdf.pages)}")
        
        for page_num, page in enumerate(pdf.pages, 1):
            text = page.extract_text()
            if text:
                all_text.append(text)
                print(f"페이지 {page_num} 읽기 완료")
    
    return '\n'.join(all_text)

# 테스트: 첫 번째 PDF 파일 읽기
pdf_path = "data/1. 2023년01회_정보처리기사필기기출문제.pdf"
full_text = read_pdf_text(pdf_path)

print(f"\n전체 텍스트 길이: {len(full_text)} 문자")
print("\n처음 1000자:")
print(full_text[:1000])

## (참고) 기본 문제 파싱 함수

아래는 답/해설 섹션이 분리되지 않은 경우의 파싱 함수입니다. 실제로는 위의 개선된 함수들을 사용하세요.

In [ ]:
def merge_questions_and_answers(questions, answers_dict):
    """문제 리스트와 답 딕셔너리를 합칩니다."""
    
    for question in questions:
        question_num = question['문제번호']
        
        if question_num in answers_dict:
            question['답'] = answers_dict[question_num]['답']
            question['해설'] = answers_dict[question_num]['해설']
        else:
            print(f"⚠ 문제 {question_num}의 답을 찾을 수 없습니다.")
    
    return questions

# 문제와 답 매칭
complete_questions = merge_questions_and_answers(questions, answers_dict)

print(f"매칭 완료: {len(complete_questions)}개 문제\n")

# 처음 3개 확인 (답 포함)
for q in complete_questions[:3]:
    print(f"\n{'='*60}")
    print(f"문제 {q['문제번호']} ({q['점수']}점)")
    print(f"{'='*60}")
    print(f"문제: {q['문제내용'][:100]}...")
    if q['코드']:
        print(f"\n코드:")
        print(q['코드'][:200] + "..." if len(q['코드']) > 200 else q['코드'])
    print(f"\n답: {q['답']}")
    if q['해설']:
        print(f"해설: {q['해설'][:150]}...")
    print()

## (참고) 기본 파싱 함수 v2

아래는 다른 방식의 파싱 함수입니다. 실제로는 위의 개선된 함수들을 사용하세요.

In [ ]:
def parse_questions_improved(questions_text):
    """문제 섹션에서 문제들을 파싱합니다 (답은 제외)."""
    questions = []
    
    # "문제 숫자" 패턴으로 분리
    pattern = r'문제\s+(\d+)\s+(.*?)(?=문제\s+\d+|$)'
    matches = re.finditer(pattern, questions_text, re.DOTALL)
    
    for match in matches:
        question_num = int(match.group(1))
        content = match.group(2).strip()
        
        # 점수 추출
        score_match = re.search(r'\((\d+)점\)', content)
        score = int(score_match.group(1)) if score_match else 0
        
        # 코드 블록 추출 (여러 언어 지원)
        code = None
        code_patterns = [
            r'((?:def|class)\s+\w+.*?)(?=답\s*[:：]|$)',  # Python
            r'((?:public|private|protected)\s+(?:static\s+)?(?:class|void|int|String).*?)(?=답\s*[:：]|$)',  # Java
            r'(#include.*?int\s+main.*?)(?=답\s*[:：]|$)',  # C/C++
        ]
        
        for code_pattern in code_patterns:
            code_match = re.search(code_pattern, content, re.DOTALL)
            if code_match:
                code = code_match.group(1).strip()
                # 코드 부분을 문제 내용에서 제거
                content = content.replace(code, '').strip()
                break
        
        # "답 :" 이후 내용 제거 (혹시 있다면)
        content = re.sub(r'답\s*[:：].*$', '', content, flags=re.MULTILINE | re.DOTALL).strip()
        
        question_data = {
            "문제번호": question_num,
            "문제내용": content,
            "코드": code,
            "점수": score,
            "답": "",
            "해설": ""
        }
        
        questions.append(question_data)
    
    return questions

# 테스트
questions = parse_questions_improved(questions_text)
print(f"파싱된 문제 개수: {len(questions)}개\n")

# 처음 2개 확인
for q in questions[:2]:
    print(f"\n{'='*50}")
    print(f"문제 {q['문제번호']} ({q['점수']}점)")
    print(f"내용: {q['문제내용'][:150]}...")
    print(f"코드: {'있음' if q['코드'] else '없음'}")

## (참고) 샘플 데이터 예시

제공하신 예시를 직접 구조화한 샘플 데이터입니다.

In [ ]:
def parse_answers(answers_text):
    """답/해설 섹션에서 각 문제의 답과 해설을 추출합니다."""
    answers_dict = {}
    
    # [문제 숫자] 패턴으로 분리
    pattern = r'\[문제\s+(\d+)\](.*?)(?=\[문제\s+\d+\]|$)'
    matches = re.finditer(pattern, answers_text, re.DOTALL)
    
    for match in matches:
        question_num = int(match.group(1))
        content = match.group(2).strip()
        
        # 답 추출 - [문제 N] 바로 다음 줄이 답인 경우가 많음
        # 또는 명시적으로 "답:" 이 있는 경우
        lines = content.split('\n')
        answer = ""
        explanation = []
        
        in_explanation = False
        
        for i, line in enumerate(lines):
            line = line.strip()
            
            # [해설] 시작
            if re.match(r'^\[해설\]', line):
                in_explanation = True
                continue
            
            # 명시적인 "답:" 패턴
            answer_match = re.match(r'^답\s*[:：]\s*(.*)', line)
            if answer_match and not answer:
                answer = answer_match.group(1).strip()
                continue
            
            # [문제 N] 바로 다음이 답인 경우 (첫 번째 비어있지 않은 줄)
            if not answer and line and not line.startswith('※') and not line.startswith('['):
                # 너무 긴 줄은 답이 아닐 가능성이 높음
                if len(line) < 100 and not re.match(r'^(public|def|class|import|from)', line):
                    answer = line
                    continue
            
            # 해설 내용 수집
            if in_explanation and line:
                explanation.append(line)
        
        answers_dict[question_num] = {
            "답": answer,
            "해설": '\n'.join(explanation) if explanation else ""
        }
    
    return answers_dict

# 테스트
if answers_text:
    answers_dict = parse_answers(answers_text)
    print(f"파싱된 답 개수: {len(answers_dict)}개\n")
    
    # 처음 3개 확인
    for num in sorted(list(answers_dict.keys())[:3]):
        print(f"\n{'='*50}")
        print(f"[문제 {num}]")
        print(f"답: {answers_dict[num]['답']}")
        explanation_preview = answers_dict[num]['해설'][:200]
        print(f"해설: {explanation_preview}..." if len(answers_dict[num]['해설']) > 200 else f"해설: {explanation_preview}")
else:
    answers_dict = {}
    print("답/해설 섹션이 없습니다.")

## (참고) JSON 파일로 저장 함수

이 함수는 위의 전체 프로세스에서 자동으로 사용됩니다.

In [ ]:
def split_questions_and_answers(text):
    """PDF 텍스트를 문제 섹션과 답/해설 섹션으로 분리합니다."""
    
    # "기출문제 정답 및 해설" 또는 유사한 패턴 찾기
    split_patterns = [
        r'기출문제\s*정답\s*및\s*해설',
        r'정답\s*및\s*해설',
        r'정답\s*해설',
        r'모범\s*답안'
    ]
    
    for pattern in split_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            questions_section = text[:match.start()].strip()
            answers_section = text[match.start():].strip()
            print(f"✓ '{pattern}' 패턴으로 섹션 분리 성공")
            print(f"  - 문제 섹션 길이: {len(questions_section)} 문자")
            print(f"  - 답/해설 섹션 길이: {len(answers_section)} 문자")
            return questions_section, answers_section
    
    print("⚠ 답/해설 섹션을 찾지 못했습니다. 전체를 문제 섹션으로 처리합니다.")
    return text, ""

# 테스트
questions_text, answers_text = split_questions_and_answers(full_text)

print("\n=== 답/해설 섹션 처음 500자 ===")
print(answers_text[:500])

## 7. 전체 프로세스 - PDF에서 JSON까지 (답/해설 포함)

In [ ]:
def pdf_to_json_complete(pdf_path, output_path):
    """PDF 파일을 읽어서 답/해설이 포함된 JSON으로 변환"""
    
    print("="*60)
    print(f"처리 중: {pdf_path}")
    print("="*60)
    
    # 1. PDF 읽기
    print("\n[1/5] PDF 읽기 중...")
    full_text = read_pdf_text(pdf_path)
    
    # 2. 문제 섹션과 답/해설 섹션 분리
    print("\n[2/5] 섹션 분리 중...")
    questions_text, answers_text = split_questions_and_answers(full_text)
    
    # 3. 문제 파싱
    print("\n[3/5] 문제 파싱 중...")
    questions = parse_questions_improved(questions_text)
    print(f"  → {len(questions)}개 문제 발견")
    
    # 4. 답/해설 파싱
    print("\n[4/5] 답/해설 파싱 중...")
    if answers_text:
        answers_dict = parse_answers(answers_text)
        print(f"  → {len(answers_dict)}개 답/해설 발견")
        
        # 5. 문제와 답 매칭
        print("\n[5/5] 문제와 답 매칭 중...")
        complete_questions = merge_questions_and_answers(questions, answers_dict)
    else:
        print("  → 답/해설 섹션 없음")
        complete_questions = questions
    
    # 6. JSON 저장
    print("\n[완료] JSON 파일 저장 중...")
    save_questions_to_json(complete_questions, output_path)
    
    return complete_questions

# 사용 예시
# questions = pdf_to_json_complete(
#     "data/1. 2023년01회_정보처리기사필기기출문제.pdf",
#     "output/2023_01회_문제.json"
# )

## 8. 모든 PDF 파일 일괄 처리

In [ ]:
def process_all_pdfs(data_folder="data", output_folder="output"):
    """data 폴더의 모든 PDF를 JSON으로 변환"""
    
    # output 폴더 생성
    os.makedirs(output_folder, exist_ok=True)
    
    # PDF 파일 목록
    pdf_files = sorted([f for f in os.listdir(data_folder) if f.endswith('.pdf')])
    print(f"발견된 PDF 파일: {len(pdf_files)}개\n")
    
    all_results = {}
    
    for idx, pdf_file in enumerate(pdf_files, 1):
        print(f"\n\n{'#'*60}")
        print(f"# [{idx}/{len(pdf_files)}] {pdf_file}")
        print(f"{'#'*60}\n")
        
        pdf_path = os.path.join(data_folder, pdf_file)
        output_name = pdf_file.replace('.pdf', '.json')
        output_path = os.path.join(output_folder, output_name)
        
        try:
            questions = pdf_to_json_complete(pdf_path, output_path)
            all_results[pdf_file] = {
                "status": "success",
                "count": len(questions),
                "output": output_path
            }
        except Exception as e:
            import traceback
            print(f"\n❌ 오류 발생: {e}")
            print(traceback.format_exc())
            all_results[pdf_file] = {
                "status": "error",
                "error": str(e)
            }
    
    # 처리 결과 요약
    print("\n\n" + "="*60)
    print("최종 처리 결과 요약")
    print("="*60)
    
    success_count = 0
    error_count = 0
    
    for pdf, result in all_results.items():
        if result["status"] == "success":
            print(f"✓ {pdf}: {result['count']}개 문제")
            success_count += 1
        else:
            print(f"✗ {pdf}: {result['error']}")
            error_count += 1
    
    print(f"\n성공: {success_count}개, 실패: {error_count}개")
    
    return all_results

# 실행
# results = process_all_pdfs()

## 9. JSON 파일 읽기 및 확인

In [ ]:
def load_and_display_json(json_path):
    """JSON 파일을 읽어서 예쁘게 출력"""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"총 문제 개수: {len(data)}\n")
    
    for i, question in enumerate(data, 1):
        print(f"\n{'='*60}")
        print(f"문제 {question.get('문제번호', i)} ({question.get('점수', 0)}점)")
        print(f"{'='*60}")
        
        # 문제 내용
        content = question.get('문제내용', '')
        print(content[:200] + "..." if len(content) > 200 else content)
        
        # 코드
        if question.get('코드'):
            print(f"\n[코드]")
            code = question.get('코드', '')
            print(code[:300] + "..." if len(code) > 300 else code)
        
        # 답
        print(f"\n답: {question.get('답', '(없음)')}")
        
        # 해설
        if question.get('해설'):
            explanation = question.get('해설', '')
            print(f"\n[해설]")
            print(explanation[:200] + "..." if len(explanation) > 200 else explanation)
        
        if i >= 3:  # 처음 3개만 출력
            print(f"\n\n... 외 {len(data) - 3}개 문제")
            break

# 사용 예시
# load_and_display_json("output/2023_01회_문제.json")

## 6. JSON 파일로 저장

In [ ]:
def save_questions_to_json(questions, output_file):
    """문제들을 JSON 파일로 저장합니다."""
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(questions, f, ensure_ascii=False, indent=2)
    print(f"저장 완료: {output_file}")
    print(f"총 {len(questions)}개 문제가 저장되었습니다.")

# 샘플 데이터 저장
save_questions_to_json(sample_questions, "output/sample_questions.json")

## 7. 전체 프로세스 - PDF에서 JSON까지

In [ ]:
def pdf_to_json(pdf_path, output_path):
    """PDF 파일을 읽어서 JSON으로 변환"""
    
    # 1. PDF 읽기
    print("PDF 읽기 중...")
    full_text = read_pdf_text(pdf_path)
    
    # 2. 문제 파싱
    print("\n문제 파싱 중...")
    questions = parse_questions(full_text)
    
    # 3. JSON 저장
    print("\nJSON 파일 저장 중...")
    save_questions_to_json(questions, output_path)
    
    return questions

# 사용 예시
# questions = pdf_to_json(
#     "data/1. 2023년01회_정보처리기사필기기출문제.pdf",
#     "output/2023_01회_문제.json"
# )

## 8. 모든 PDF 파일 일괄 처리

In [ ]:
def process_all_pdfs(data_folder="data", output_folder="output"):
    """data 폴더의 모든 PDF를 JSON으로 변환"""
    
    # output 폴더 생성
    os.makedirs(output_folder, exist_ok=True)
    
    # PDF 파일 목록
    pdf_files = [f for f in os.listdir(data_folder) if f.endswith('.pdf')]
    print(f"발견된 PDF 파일: {len(pdf_files)}개\n")
    
    all_results = {}
    
    for pdf_file in pdf_files:
        print(f"{'='*60}")
        print(f"처리 중: {pdf_file}")
        print(f"{'='*60}")
        
        pdf_path = os.path.join(data_folder, pdf_file)
        output_name = pdf_file.replace('.pdf', '.json')
        output_path = os.path.join(output_folder, output_name)
        
        try:
            questions = pdf_to_json(pdf_path, output_path)
            all_results[pdf_file] = {
                "status": "success",
                "count": len(questions),
                "output": output_path
            }
        except Exception as e:
            print(f"오류 발생: {e}")
            all_results[pdf_file] = {
                "status": "error",
                "error": str(e)
            }
        
        print("\n")
    
    # 처리 결과 요약
    print("\n" + "="*60)
    print("처리 완료 요약")
    print("="*60)
    for pdf, result in all_results.items():
        if result["status"] == "success":
            print(f"✓ {pdf}: {result['count']}개 문제")
        else:
            print(f"✗ {pdf}: {result['error']}")
    
    return all_results

# 실행
# results = process_all_pdfs()

## 9. JSON 파일 읽기 및 확인

In [ ]:
def load_and_display_json(json_path):
    """JSON 파일을 읽어서 예쁘게 출력"""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"총 문제 개수: {len(data)}\n")
    
    for i, question in enumerate(data, 1):
        print(f"\n{'='*60}")
        print(f"문제 {question.get('문제번호', i)} ({question.get('점수', 0)}점)")
        print(f"{'='*60}")
        print(question.get('문제내용', '')[:200] + "...")
        if question.get('코드'):
            print(f"\n[코드 포함]")
        print(f"\n답: {question.get('답', '(없음)')}")
        
        if i >= 3:  # 처음 3개만 출력
            print(f"\n... 외 {len(data) - 3}개 문제")
            break

# 사용 예시
# load_and_display_json("output/sample_questions.json")